Run these first to set up the categories for the clusterers.

In [3]:
import registryModeling
import csv

def inspectClusters(predictions, products, numClusters = 10):
    for j in range(numClusters):
        for i in range(len(predictions)):
            if predictions[i] ==j:
                print '%d : %s' % (j, products[i])


In [5]:
elecs = []

with open('../Target-Product-Feed-All-Items.csv', 'rU') as target:
    reader = csv.DictReader(target)
    for r in reader:
        if r['Category'].startswith('Electronics'):
            elecs.append(r['Category'])

In [6]:
#tracked = ['Jewelry', 'Garage', 'Gifts', 'Apparel', 'Home', 'Electronics', 'Kitchen', 'Patio', 'Furniture']
elecSubs = list(set([e.split(' > ')[1] for e in elecs]))
tracked = ['Jewelry', 'Garage', 'Gifts', 'Apparel', 'Home', 'Kitchen', 'Patio', 'Furniture', 'Electronics'] + elecSubs
print elecSubs

['Vehicle_mobile_handheld', 'Television', 'Audio', 'Gps And Navigation', 'Music - Mmbv', 'Camera And Optics', 'Movies - Mmbv', 'Electronics - Miscellaneous', 'Musical Instruments', 'Computer', 'Video', 'Home Office Electronics', 'Cell Phones And Telephones', 'Video Game', 'Books - Mmbv']


In [7]:
#import unicodecsv
from collections import defaultdict

targetProducts = defaultdict(list)

with open('../Target-Product-Feed-All-Items.csv', 'rU') as target:
    reader = csv.DictReader(target)
    try:
        for r in reader:
            for prefix in tracked:
                if r['Category'].startswith(prefix):
                    try:
                        targetProducts[prefix].append(unicode(r['Product Name']))
                        #targetProducts[prefix].append(r['Product Name'])
                    except UnicodeDecodeError:
                        print r['Product Name'] #just ditching unicode errors right now
                        #raise
            if r['Category'].startswith('Electronics'):
                subCat = r['Category'].split(' > ')[1]
                try:
                    targetProducts[subCat].append(unicode(r['Product Name']))
                except UnicodeDecodeError:
                    print r['Product Name']

    except UnicodeDecodeError:
        print r
        raise

Lush DǸcor Belle Vertical Texture Shower Curtain - Cream
Lush DǸcor Large Ruffle Shower Curtain
Lilyette by Bali�� Women's Minimizer with Lace Bra Black
Le Pi�� Belle Canzoni di Gianni Bella (Greatest Hits)
Le Pi�� Belle Canzoni di Gianni Bella (Greatest Hits)
Tom Brady Wall DǸcor
Puccini: Highlights from La Boh��me & Madama Butterfly (Karaoke, Mix Album)
Puccini: Highlights from La Boh��me & Madama Butterfly (Karaoke, Mix Album)
Richard Wagner: Der fliegende Holl��nder (Bayreuth, 1961)
Richard Wagner: Der fliegende Holl��nder (Bayreuth, 1961)
De Noche: Clǭsicos a Mi Manera
De Noche: Clǭsicos a Mi Manera
Johannes Brahms: H��ndel Variations, Op. 24; Max Reger: Bach Variations, Op. 81
Johannes Brahms: H��ndel Variations, Op. 24; Max Reger: Bach Variations, Op. 81
Lush Decor�� Rosina Comforter Set - Ivory (Queen)
Generaci��n Tribal
Generaci��n Tribal
Lilyette by Bali�� Women's Minimizer with Lace Bra Pearl
Tarǭntula
Tarǭntula
Mr. Coffee CafǸ Barista Espresso Maker -Stainless Steel/Black
R

In [9]:
for k in targetProducts:
    print k + ': ' + str(len(targetProducts[k]))

Jewelry: 27778
Furniture: 24439
Music - Mmbv: 77439
Camera And Optics: 661
Electronics - Miscellaneous: 3898
Cell Phones And Telephones: 782
Electronics: 258325
Gps And Navigation: 37
Gifts: 3177
Video Game: 2166
Kitchen: 7178
Television: 151
Movies - Mmbv: 62789
Garage: 3772
Musical Instruments: 257
Home Office Electronics: 70
Apparel: 176170
Audio: 1505
Patio: 12909
Vehicle_mobile_handheld: 49
Computer: 1447
Video: 30
Home: 69793
Books - Mmbv: 107044


Now the product names are organized by some wacky categories (Electronics was over 200,000 products, so I broke down the second level categories).  Each category (as I'm currently slurping them) gets a clusterer which will eventually be tuned to its size and other (ugh) intangibles that experimentation will help nail down.  Very data dependent.

These clusterers are all arbitrarily set to 100 centroids because that was a level that let them all fit in memory concurrently.  Not currently sure how to serialize or otherwise minimize memory footprint.

In [ ]:
import registryModeling
targetClusterers = {k: None for k in tracked}

for cat in targetClusterers:
    targetClusterers[cat] = registryModeling.clusterWithTFIDF(targetProducts[cat], len(targetProducts[cat])/3) # len(targetProducts)/11) #stab in the dark

Next step is to take a user, convert them to an array containing their Target registry items (product names, categories, and product ids), and run them through the clusterers to determine how many instances of each cluster they have.  See the readme for user query stuff.

Then, with those results as vectors, either do nearest neighbor analysis to get sets of similar users to share with LJ or whatever matrix completion technique tickles your fancy.